In [8]:
import pandas as pd
import seaborn as sb
import numpy as np
from matplotlib import pyplot as pyplot
import plotly.express as px

pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.width', 10000)

FUNCIONES

In [27]:
def dame_variables_categoricas(dataset):
    '''
    ----------------------------------------------------------------------------------------------------------
    Función clasificar_variables:
    ----------------------------------------------------------------------------------------------------------
        - Descripción : Funcion que recibe un dataset y devuelve una lista respectiva para cada tipo de variable
        (Categórica, Continua, Booleana y No clasificada)
        - Inputs:
            -- dataset : Pandas dataframe que contiene los datos
        - Return : 
            -- 1: la ejecución es incorrecta
            -- lista_var_bool: lista con los nombres de las variables booleanas del dataset de entrada, con valores
            unicos con una longitud de dos, que sean del tipo booleano y que presenten valores 'yes','no','n' & 'y' .
            -- lista_var_cat: lista con los nombres de las variables categóricas del dataset de entrada, con valores
            de tipo object o tipo categorical.
            -- lista_var_con: lista con los nombres de las variables continuas del dataset de entrada, con valores 
            de tipo float o con una longitud de valores unicos mayor a dos. 
            -- lista_var_no_clasificadas: lista con los nombres de las variables no clasificadas del dataset de 
            entrada, que no cumplen con los aspectos anteriormente mencionadas de las demás listas. 
    '''
    
    if dataset is None:
        # Resultante al no brindar ningun DataFrame
        print(u'\nFaltan argumentos por pasar a la función')
        return 1
    
    # Listas para cada tipo de variable
    lista_var_bool = []
    lista_var_cat = []
    lista_var_con = []
    lista_var_no_clasificadas = []
    
    for columna in dataset.columns:
        # Valores unicos por columna sin los NAs
        valores_unicos = dataset[columna].dropna().unique()
        # Trato de mayusculas
        valores_lower = set(val.lower() for val in valores_unicos if isinstance(val, str))
        
        # Variables booleanas
        if (len(valores_unicos) == 2 and
            (valores_lower <= {"yes", "no", "n", "y"} or
             set(valores_unicos) <= {0, 1} or 
             pd.api.types.is_bool_dtype(dataset[columna]))):
            lista_var_bool.append(columna)
        
        # Variables continuas
        elif pd.api.types.is_float_dtype(dataset[columna]) and len(valores_unicos) > 2:
            lista_var_con.append(columna)
        
        # Variables categóricas
        elif pd.api.types.is_object_dtype(dataset[columna]) or pd.api.types.is_categorical_dtype(dataset[columna]):
            lista_var_cat.append(columna)
        
        elif set(valores_unicos).issubset({1, 2, 3}):
            lista_var_cat.append(columna)
        
        # Variables no clasificadas
        else:
            lista_var_no_clasificadas.append(columna) 

    # Calcula la cantidad de cada tipo de variable
    c_v_b = len(lista_var_bool)
    c_v_ca = len(lista_var_cat)
    c_v_co = len(lista_var_con)
    c_v_f = len(lista_var_no_clasificadas)

    print("Variables Booleanas:", c_v_b, lista_var_bool)
    print('============================================================================================================================================================================')
    print("Variables Categóricas:", c_v_ca, lista_var_cat)
    print('============================================================================================================================================================================')
    print("Variables Continuas:", c_v_co, lista_var_con)
    print('============================================================================================================================================================================')
    print("Variables no clasificadas:", c_v_f, lista_var_no_clasificadas)

    return lista_var_bool, lista_var_cat, lista_var_con, lista_var_no_clasificadas

APPLICATION DATA

Nos encontramos ante un problema de aprendizaje supervisado, ya que contamos con una variable objetivo, TARGET, que clasifica a los clientes según si pagan su préstamo (valor 0) o no (valor 1). En este análisis exploratorio de datos (EDA) investigaremos el dataset application_data.csv para comprender las características y comportamientos de las variables, identificando patrones, distribuciones y relaciones clave. Además, abordaremos posibles valores atípicos, sesgos y valores faltantes que puedan afectar el rendimiento de futuros modelos predictivos.

Este análisis tiene como objetivo principal ayudar al banco a tomar decisiones más informadas, reduciendo el riesgo de impagos al identificar perfiles de clientes con mayor probabilidad de incumplimiento, sin perjudicar a quienes cumplen; a la vez que también reduce la probabilidad de dejar pasar a clientes que no incumplirán. El modelo se basará en datos proporcionados por el cliente al solicitar el préstamo, como información demográfica, financiera y laboral, y será evaluado mediante métricas clave como la curva ROC, AUC, precisión y recall.

Los objetivos específicos de este EDA son:

- Analizar las distribuciones de las variables categóricas y numéricas, explorando su relación con la variable objetivo.
- Identificar y tratar valores nulos o atípicos para mejorar la calidad de los datos.
- Explorar relaciones clave entre las variables y su impacto en el objetivo.
- Proponer transformaciones y selecciones de variables para optimizar el dataset para el modelado.APPLICATION DATA



ANÁLISIS GENERAL DE LA TABLA Y PROCESAMIENTO INICIAL

In [10]:
path_folder = "C:/Users/Guille/OneDrive/Desktop/Aprendizaje automático/Practica_EDA/data/"
pd_loan = pd.read_csv(path_folder + "application_data.csv", low_memory = False)

pd_loan.head()



,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [11]:

print(pd_loan.shape, pd_loan.drop_duplicates().shape)

(307511, 122) (307511, 122)


Nos encontramos ante un data frame de 122 columnas (variables) y 307511 observaciones

In [12]:

pd_loan.dtypes.to_dict()


{'SK_ID_CURR': dtype('int64'),
 'TARGET': dtype('int64'),
 'NAME_CONTRACT_TYPE': dtype('O'),
 'CODE_GENDER': dtype('O'),
 'FLAG_OWN_CAR': dtype('O'),
 'FLAG_OWN_REALTY': dtype('O'),
 'CNT_CHILDREN': dtype('int64'),
 'AMT_INCOME_TOTAL': dtype('float64'),
 'AMT_CREDIT': dtype('float64'),
 'AMT_ANNUITY': dtype('float64'),
 'AMT_GOODS_PRICE': dtype('float64'),
 'NAME_TYPE_SUITE': dtype('O'),
 'NAME_INCOME_TYPE': dtype('O'),
 'NAME_EDUCATION_TYPE': dtype('O'),
 'NAME_FAMILY_STATUS': dtype('O'),
 'NAME_HOUSING_TYPE': dtype('O'),
 'REGION_POPULATION_RELATIVE': dtype('float64'),
 'DAYS_BIRTH': dtype('int64'),
 'DAYS_EMPLOYED': dtype('int64'),
 'DAYS_REGISTRATION': dtype('float64'),
 'DAYS_ID_PUBLISH': dtype('int64'),
 'OWN_CAR_AGE': dtype('float64'),
 'FLAG_MOBIL': dtype('int64'),
 'FLAG_EMP_PHONE': dtype('int64'),
 'FLAG_WORK_PHONE': dtype('int64'),
 'FLAG_CONT_MOBILE': dtype('int64'),
 'FLAG_PHONE': dtype('int64'),
 'FLAG_EMAIL': dtype('int64'),
 'OCCUPATION_TYPE': dtype('O'),
 'CNT_FAM_MEMB

EXPLORACIÓN DE LA VARIABLE OBJETIVO Y TRATAMIENTO

In [13]:
pd_plot_target = pd_loan['TARGET']\
        .value_counts(normalize=True)\
        .mul(100).rename('percent').reset_index()

pd_plot_target_conteo = pd_loan['TARGET'].value_counts().reset_index()
pd_plot_target_pc = pd.merge(pd_plot_target, pd_plot_target_conteo, on = ['TARGET'], how='inner')

print(pd_plot_target_pc)

   TARGET    percent   count
0       0  91.927118  282686
1       1   8.072882   24825


In [14]:
fig = px.histogram(pd_plot_target_pc, x="TARGET", y=['percent'])
fig.update_xaxes(tickvals=[0, 1])
fig.show()

In [15]:
pd_series_null_columns = pd_loan.isnull().sum().sort_values(ascending=False)
pd_series_null_rows = pd_loan.isnull().sum(axis=1).sort_values(ascending=False)
print(pd_series_null_columns.shape, pd_series_null_rows.shape)

pd_null_columnas = pd.DataFrame(pd_series_null_columns, columns=['nulos_columnas'])     
pd_null_filas = pd.DataFrame(pd_series_null_rows, columns=['nulos_filas'])  
pd_null_filas['TARGET'] = pd_loan['TARGET'].copy()
pd_null_columnas['porcentaje_columnas'] = pd_null_columnas['nulos_columnas']/pd_loan.shape[0]
pd_null_filas['porcentaje_filas']= pd_null_filas['nulos_filas']/pd_loan.shape[1]

(122,) (307511,)


In [16]:
pd_null_columnas

,nulos_columnas,porcentaje_columnas
COMMONAREA_MEDI,214865,0.698723
COMMONAREA_AVG,214865,0.698723
COMMONAREA_MODE,214865,0.698723
NONLIVINGAPARTMENTS_MODE,213514,0.694330
NONLIVINGAPARTMENTS_AVG,213514,0.694330
NONLIVINGAPARTMENTS_MEDI,213514,0.694330
FONDKAPREMONT_MODE,210295,0.683862
LIVINGAPARTMENTS_MODE,210199,0.683550
LIVINGAPARTMENTS_AVG,210199,0.683550
LIVINGAPARTMENTS_MEDI,210199,0.683550


In [17]:
threshold=0.8
list_vars_not_null = list(pd_null_columnas[pd_null_columnas['porcentaje_columnas']<threshold].index)
pd_loan_filter_null = pd_loan.loc[:, list_vars_not_null]
pd_loan_filter_null.shape

(307511, 122)

Aqui, aplicamos un umbral para eliminar todas aquellas columnas que posean más del 80% de valores nulos. En este caso, no prescindimos de ninguna columna y seguiremos
utilizando las iniciales, que son 122

In [18]:
pd_null_filas

,nulos_filas,TARGET,porcentaje_filas
185713,61,0,0.5
133770,61,0,0.5
197736,61,0,0.5
116937,61,0,0.5
269492,61,0,0.5
...,...,...,...
129942,0,0,0.0
129929,0,0,0.0
129924,0,0,0.0
129911,0,0,0.0


TIPOS: VARIABLES CATEGÓRICAS Y NUMÉRICAS


In [28]:
list_cat_vars, other = dame_variables_categoricas(dataset=pd_loan_filter_null)
pd_loan_filter_null[list_cat_vars] = pd_loan_filter_null[list_cat_vars].astype("category")
pd_loan_filter_null[list_cat_vars].head()

C:\Users\Guille\AppData\Local\Temp\ipykernel_5244\3774187972.py:51: DeprecationWarning:

is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead



Variables Booleanas: 36 ['EMERGENCYSTATE_MODE', 'FLAG_DOCUMENT_8', 'FLAG_OWN_CAR', 'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_21', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 'FLAG_OWN_REALTY', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_12', 'FLAG_PHONE', 'LIVE_CITY_NOT_WORK_CITY', 'REG_CITY_NOT_WORK_CITY', 'TARGET', 'REG_CITY_NOT_LIVE_CITY', 'LIVE_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_WORK_REGION', 'REG_REGION_NOT_LIVE_REGION', 'FLAG_EMAIL', 'FLAG_CONT_MOBILE', 'FLAG_WORK_PHONE', 'FLAG_EMP_PHONE', 'FLAG_MOBIL']
Variables Categóricas: 18 ['FONDKAPREMONT_MODE', 'WALLSMATERIAL_MODE', 'HOUSETYPE_MODE', 'OCCUPATION_TYPE', 'NAME_TYPE_SUITE', 'CNT_CHILDREN', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'HOUR_APPR_PROCESS_START', 'WEEKDAY_APPR_PROCESS_START', 'REGION_R

ValueError: too many values to unpack (expected 2)

In [20]:
pd_loan_filter_null[list_cat_vars].dtypes
print(pd_loan_filter_null[list_cat_vars])

       FONDKAPREMONT_MODE WALLSMATERIAL_MODE  HOUSETYPE_MODE EMERGENCYSTATE_MODE OCCUPATION_TYPE NAME_TYPE_SUITE CNT_CHILDREN FLAG_DOCUMENT_8 NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_DOCUMENT_2 FLAG_DOCUMENT_3 FLAG_DOCUMENT_4 FLAG_DOCUMENT_5 FLAG_DOCUMENT_6 FLAG_DOCUMENT_7 FLAG_DOCUMENT_9 FLAG_DOCUMENT_21 FLAG_DOCUMENT_10 FLAG_DOCUMENT_11 FLAG_OWN_REALTY FLAG_DOCUMENT_13 FLAG_DOCUMENT_14 FLAG_DOCUMENT_15 FLAG_DOCUMENT_16 FLAG_DOCUMENT_17 FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG_DOCUMENT_12 FLAG_PHONE LIVE_CITY_NOT_WORK_CITY REG_CITY_NOT_WORK_CITY TARGET REG_CITY_NOT_LIVE_CITY LIVE_REGION_NOT_WORK_REGION REG_REGION_NOT_WORK_REGION REG_REGION_NOT_LIVE_REGION HOUR_APPR_PROCESS_START WEEKDAY_APPR_PROCESS_START REGION_RATING_CLIENT_W_CITY REGION_RATING_CLIENT FLAG_EMAIL FLAG_CONT_MOBILE       ORGANIZATION_TYPE FLAG_WORK_PHONE FLAG_EMP_PHONE FLAG_MOBIL  NAME_HOUSING_TYPE    NAME_FAMILY_STATUS            NAME_EDUCATION_TYPE      NAME_INCOME_TYPE
0        reg oper accoun

PROCESAMIENTO INICIAL DE ALGUNAS VARIABLES

In [22]:
dia = { "MONDAY": 1, "TUESDAY": 2, "WEDNESDAY": 3, "THURSDAY": 4, "FRIDAY": 5, "SATURDAY": 6, "SUNDAY": 7}

pd_loan_filter_null['NWEEKDAY_PROCESS_START'] = pd_loan_filter_null['WEEKDAY_APPR_PROCESS_START'].replace(dia).astype("category")



C:\Users\Guille\AppData\Local\Temp\ipykernel_5244\560063872.py:3: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

C:\Users\Guille\AppData\Local\Temp\ipykernel_5244\560063872.py:3: FutureWarning:

The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.



In [23]:
pd_loan_filter_null.head()


,COMMONAREA_MEDI,COMMONAREA_AVG,COMMONAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,FONDKAPREMONT_MODE,LIVINGAPARTMENTS_MODE,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,FLOORSMIN_AVG,FLOORSMIN_MODE,FLOORSMIN_MEDI,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,YEARS_BUILD_AVG,OWN_CAR_AGE,LANDAREA_MEDI,LANDAREA_MODE,LANDAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_AVG,BASEMENTAREA_MODE,EXT_SOURCE_1,NONLIVINGAREA_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,ELEVATORS_MEDI,ELEVATORS_AVG,ELEVATORS_MODE,WALLSMATERIAL_MODE,APARTMENTS_MEDI,APARTMENTS_AVG,APARTMENTS_MODE,ENTRANCES_MEDI,ENTRANCES_AVG,ENTRANCES_MODE,LIVINGAREA_AVG,LIVINGAREA_MODE,LIVINGAREA_MEDI,HOUSETYPE_MODE,FLOORSMAX_MODE,FLOORSMAX_MEDI,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_AVG,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OCCUPATION_TYPE,EXT_SOURCE_3,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_TYPE_SUITE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CNT_CHILDREN,FLAG_DOCUMENT_8,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_9,FLAG_DOCUMENT_21,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_OWN_REALTY,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_12,AMT_CREDIT,AMT_INCOME_TOTAL,FLAG_PHONE,LIVE_CITY_NOT_WORK_CITY,REG_CITY_NOT_WORK_CITY,TARGET,REG_CITY_NOT_LIVE_CITY,LIVE_REGION_NOT_WORK_REGION,REG_REGION_NOT_WORK_REGION,REG_REGION_NOT_LIVE_REGION,HOUR_APPR_PROCESS_START,WEEKDAY_APPR_PROCESS_START,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,FLAG_EMAIL,FLAG_CONT_MOBILE,ORGANIZATION_TYPE,FLAG_WORK_PHONE,FLAG_EMP_PHONE,FLAG_MOBIL,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_EMPLOYED,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_HOUSING_TYPE,NAME_FAMILY_STATUS,NAME_EDUCATION_TYPE,NAME_INCOME_TYPE,SK_ID_CURR,NWEEKDAY_PROCESS_START
0,0.0144,0.0143,0.0144,0.0,0.0000,0.0000,reg oper account,0.022,0.0202,0.0205,0.1250,0.1250,0.1250,0.6243,0.6341,0.6192,NaN,0.0375,0.0377,0.0369,0.0369,0.0369,0.0383,0.083037,0.0,0.0000,0.00,0.00,0.00,0.0000,"Stone, brick",0.0250,0.0247,0.0252,0.0690,0.0690,0.0690,0.0190,0.0198,0.0193,block of flats,0.0833,0.0833,0.0833,0.9722,0.9722,0.9722,0.0149,No,Laborers,0.139376,0.0,0.0,0.0,0.0,0.0,1.0,Unaccompanied,2.0,2.0,2.0,2.0,0.262949,351000.0,24700.5,1.0,-1134.0,0,0,Cash loans,M,N,0,1,0,0,0,0,0,0,0,0,Y,0,0,0,0,0,0,0,0,0,406597.5,202500.0,1,0,0,1,0,0,0,0,10,WEDNESDAY,2,2,0,1,Business Entity Type 3,0,1,1,-2120,-3648.0,-637,-9461,0.018801,House / apartment,Single / not married,Secondary / secondary special,Working,100002,3
1,0.0608,0.0605,0.0497,0.0,0.0039,0.0039,reg oper account,0.079,0.0773,0.0787,0.3333,0.3333,0.3333,0.7987,0.8040,0.7960,NaN,0.0132,0.0128,0.0130,0.0529,0.0529,0.0538,0.311267,0.0,0.0098,0.01,0.08,0.08,0.0806,Block,0.0968,0.0959,0.0924,0.0345,0.0345,0.0345,0.0549,0.0554,0.0558,block of flats,0.2917,0.2917,0.2917,0.9851,0.9851,0.9851,0.0714,No,Core staff,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Family,1.0,0.0,1.0,0.0,0.622246,1129500.0,35698.5,2.0,-828.0,0,0,Cash loans,F,N,0,1,0,0,0,0,0,0,0,0,N,0,0,0,0,0,0,0,0,0,1293502.5,270000.0,1,0,0,0,0,0,0,0,11,MONDAY,1,1,0,1,School,0,1,1,-291,-1186.0,-1188,-16765,0.003541,House / apartment,Married,Higher education,State servant,100003,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Laborers,0.729567,0.0,0.0,0.0,0.0,0.0,0.0,Unaccompanied,0.0,0.0,0.0,0.0,0.555912,135000.0,6750.0,1.0,-815.0,0,0,Revolving loans,M,Y,0,0,0,0,0,0,0,0,0,0,Y,0,0,0,0,0,0,0,0,0,135000.0

In [24]:
pd_loan_filter_null.shape

(307511, 123)

In [25]:
pd_loan_filter_null.to_csv("../data/pd_data_preprocessing.csv")